In [ ]:
 import pandas as pd
df = pd.read_csv('/content/HateSpeech_Kenya.csv')
# Display the first few rows of the dataframe
df.head()

,hate_speech,offensive_language,neither,Class,Tweet
0,0,0,3,0,['The political elite are in desperation. Ordi...
1,0,0,3,0,"[""Am just curious the only people who are call..."
2,0,0,3,0,['USERNAME_3 the area politicians are the one ...
3,0,0,3,0,['War expected in Nakuru if something is not d...
4,0,0,3,0,['USERNAME_4 tells kikuyus activists that they...


We need to import the necessary libraries inorder to be able to perfom our task

So we begin by importing all that is needed


In [ ]:
import pandas as pd
import numpy as np
import re
import string
import nltk
import spacy
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score
import gensim
from gensim.models import Word2Vec, KeyedVectors
import pickle

# Download necessary NLTK data
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

We then view and explore the dataset

In [ ]:
nlp = spacy.load('en_core_web_sm')
print(df.columns)
print(df.head())
print(df.info())
print(df['Class'].value_counts())

text_column = 'Tweet'
label_column = 'Class'

print()

Index(['hate_speech', 'offensive_language', 'neither', 'Class', 'Tweet'], dtype='object')
   hate_speech  offensive_language  neither  Class  \
0            0                   0        3      0   
1            0                   0        3      0   
2            0                   0        3      0   
3            0                   0        3      0   
4            0                   0        3      0   

                                               Tweet  
0  ['The political elite are in desperation. Ordi...  
1  ["Am just curious the only people who are call...  
2  ['USERNAME_3 the area politicians are the one ...  
3  ['War expected in Nakuru if something is not d...  
4  ['USERNAME_4 tells kikuyus activists that they...  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48076 entries, 0 to 48075
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   hate_speech         48076 non-null  int64 
 1 

We then begin text processsing

In [ ]:
import re
import string
import nltk
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import stopwords

#Removing any missing values
df.dropna(subset=['Tweet'], inplace=True)


contractions = {
    "can't": "cannot",
    "won't": "will not",
    "n't": " not",
    "'re": " are",
    "'s": " is",
    "'d": " would",
    "'ll": " will",
    "'t": " not",
    "'ve": " have",
    "'m": " am"
}

def expand_contractions(text, contractions_dict=contractions):
    for key, value in contractions_dict.items():
        text = text.replace(key, value)
    return text


def preprocess_text(text):
    text = BeautifulSoup(text, "html.parser").get_text()  # This is to remove any HTML tags
    text = text.lower()  # Converts texts into lowercase
    text = expand_contractions(text)
    text = re.sub(f"[{string.punctuation}]", "", text)  # Removes punctuation in the text
    text = re.sub("\d+", "", text)  #Removes numbers
    text = text.split()
    text = [word for word in text if word not in stopwords.words('english')]  # Removes all stopwords
    text = " ".join(text)  #
    return text

# Applying preprocessing
df['cleaned_text'] = df['Tweet'].apply(preprocess_text)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
<ipython-input-9-81f07d9520c8>:32: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text, "html.parser").get_text()  # This is to remove any HTML tags


Perfomring tokenization,stemming, POS tagging and lematization.

In [ ]:
def tokenize_text(text):
    return word_tokenize(text)

def stem_text(text):
    stemmer = PorterStemmer()
    return [stemmer.stem(word) for word in text]

def pos_tag_text(text):
    return nltk.pos_tag(text)

def lemmatize_text(text):
    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(word) for word in text]


df['tokens'] = df['cleaned_text'].apply(tokenize_text)
df['stemmed_tokens'] = df['tokens'].apply(stem_text)
df['pos_tags'] = df['tokens'].apply(pos_tag_text)
df['lemmatized_tokens'] = df['tokens'].apply(lemmatize_text)


le = LabelEncoder()
df['encoded_label'] = le.fit_transform(df[label_column])


In [ ]:
from sklearn.model_selection import train_test_split


X = df['cleaned_text']
y = df['encoded_label']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=5000)

# Fit and transform the data
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

Then we start building our linear regression model

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

#Initializing and training the model
model = LogisticRegression()
model.fit(X_train_tfidf, y_train)


y_pred = model.predict(X_test_tfidf)


#Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy: 0.7476081530782029
              precision    recall  f1-score   support

           0       0.77      0.97      0.86      7166
           1       0.43      0.11      0.17      1806
           2       0.37      0.11      0.16       644

    accuracy                           0.75      9616
   macro avg       0.52      0.39      0.40      9616
weighted avg       0.68      0.75      0.68      9616



/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
import pickle


with open('hate_speech_model.pkl', 'wb') as file:
    pickle.dump(model, file)
with open('tfidf_vectorizer.pkl', 'wb') as file:
    pickle.dump(vectorizer, file)




with open('hate_speech_model.pkl', 'rb') as file:
    loaded_model = pickle.load(file)

with open('tfidf_vectorizer.pkl', 'rb') as file:
    loaded_vectorizer = pickle.load(file)

#Making predictions using the loaded model and vectorizer
sample_text = ["This is a test tweet"]
sample_text_tfidf = loaded_vectorizer.transform(sample_text)
predicted_class = loaded_model.predict(sample_text_tfidf)
print(predicted_class)

[0]


After perfoming the required cleaning steps(Tokenization,removing punctuation etc) and making sure the logistic regression model is accruate we then view the classified tweets which were singled out from the rest of the other tweets.

In [ ]:

df_test = pd.DataFrame({
    text_column: df.loc[X_test.index, text_column],
    'cleaned_text': X_test,
    'predicted_class': y_pred
})

#To the classified tweets
print(df_test)

                                                   Tweet  \
465    ['USERNAME_417 on something about religion and...   
22169  ['Kenya belongs to Kikuyus & Kalenjins all oth...   
20879  ["This is one of my fav weddings even though i...   
26268    ['who are those colourful white luos ....?!!!']   
2875   ['Well well well chinese say #GenghisKhan is c...   
...                                                  ...   
47601  ['Those accusing Luos of Migori hooliganism.Wh...   
30139  ['Bei ya unga na maziwa ziko uko juu us luhyas...   
15381  ['When it comes to crossing the Moroccan deser...   
7020   ['Luhya tweet~ RTp USERNAME_1514: am the type ...   
38508  ['Kikuyus are in every corner of the republic ...   

                                            cleaned_text  predicted_class  
465                    username something religion jokes                0  
22169  kenya belongs kikuyus kalenjins others slaves ...                2  
20879  one fav weddings even though straight photog